# 🚀 Multi-Agent Platform: Bedrock Setup & Testing

This notebook sets up the environment and tests Amazon Bedrock connectivity for our Multi-Agent Orchestration Platform.

## What we'll accomplish:
1. Install required packages
2. Configure AWS credentials and region
3. Test Bedrock model access
4. Create base agent framework
5. Test individual models (Claude 3 Sonnet, Haiku, Titan)

## 📦 Install Required Packages

In [ ]:
# Install required packages
!pip install langchain langchain-aws boto3 redis pandas numpy matplotlib seaborn
!pip install fastapi uvicorn python-dotenv httpx

## 🔧 Environment Setup

In [ ]:
import boto3
import json
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

# Configure AWS region
AWS_REGION = 'us-east-1'
os.environ['AWS_DEFAULT_REGION'] = AWS_REGION

print(f"✅ Environment configured for region: {AWS_REGION}")
print(f"✅ Current time: {datetime.now()}")

## 🔐 AWS Credentials Check

In [ ]:
# Check AWS credentials and permissions
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    
    print("✅ AWS Credentials Valid")
    print(f"Account ID: {identity['Account']}")
    print(f"User ARN: {identity['Arn']}")
    
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("Please ensure your SageMaker execution role has proper permissions")

## 🤖 Test Bedrock Model Access

In [ ]:
# Initialize Bedrock client
bedrock = boto3.client('bedrock', region_name=AWS_REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)

# List available foundation models
try:
    models = bedrock.list_foundation_models()
    
    print("🤖 Available Bedrock Models:")
    print("=" * 50)
    
    for model in models['modelSummaries']:
        print(f"Model ID: {model['modelId']}")
        print(f"Provider: {model['providerName']}")
        print(f"Name: {model['modelName']}")
        print("-" * 30)
        
except Exception as e:
    print(f"❌ Error listing models: {e}")

## 🧪 Test Claude 3 Sonnet (Market Research Agent)

In [ ]:
def test_claude_sonnet(prompt):
    """Test Claude 3 Sonnet model"""
    try:
        body = json.dumps({
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 1000,
            "temperature": 0.7,
            "anthropic_version": "bedrock-2023-05-31"
        })
        
        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['content'][0]['text']
        
    except Exception as e:
        return f"Error: {e}"

# Test market research prompt
market_prompt = """
Analyze the smartphone market in Germany. Include:
1. Market size and growth rate
2. Top 3 competitors and their market share
3. Key consumer trends
4. Opportunities for a new entrant

Provide specific data and actionable insights.
"""

print("🧪 Testing Claude 3 Sonnet for Market Research...")
print("=" * 60)

sonnet_result = test_claude_sonnet(market_prompt)
print(sonnet_result)

if "Error" not in sonnet_result:
    print("\n✅ Claude 3 Sonnet test successful!")
else:
    print("\n❌ Claude 3 Sonnet test failed!")

## ⚡ Test Claude 3 Haiku (Risk Assessment Agent)

In [ ]:
def test_claude_haiku(prompt):
    """Test Claude 3 Haiku model"""
    try:
        body = json.dumps({
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 500,
            "temperature": 0.3,
            "anthropic_version": "bedrock-2023-05-31"
        })
        
        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-3-haiku-20240307-v1:0",
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['content'][0]['text']
        
    except Exception as e:
        return f"Error: {e}"

# Test risk assessment prompt
risk_prompt = """
Rate the political risk (1-10 scale) for a technology company entering the Brazilian market. Consider:
- Government stability
- Regulatory environment
- Policy consistency
- International relations

Provide the rating and 3 key risk factors.
"""

print("⚡ Testing Claude 3 Haiku for Risk Assessment...")
print("=" * 60)

haiku_result = test_claude_haiku(risk_prompt)
print(haiku_result)

if "Error" not in haiku_result:
    print("\n✅ Claude 3 Haiku test successful!")
else:
    print("\n❌ Claude 3 Haiku test failed!")

## 🏗️ Create Base Agent Framework

In [ ]:
class SageMakerBedrockAgent:
    """Base agent class for SageMaker + Bedrock implementation"""
    
    def __init__(self, agent_id: str, model_id: str, region: str = 'us-east-1'):
        self.agent_id = agent_id
        self.model_id = model_id
        self.region = region
        self.bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)
        
        print(f"✅ Initialized {agent_id} with model {model_id}")
    
    def invoke_model(self, prompt: str, max_tokens: int = 1000, temperature: float = 0.7):
        """Invoke Bedrock model with prompt"""
        try:
            if "claude" in self.model_id:
                body = json.dumps({
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": max_tokens,
                    "temperature": temperature,
                    "anthropic_version": "bedrock-2023-05-31"
                })
            else:  # Titan or other models
                body = json.dumps({
                    "inputText": prompt,
                    "textGenerationConfig": {
                        "maxTokenCount": max_tokens,
                        "temperature": temperature
                    }
                })
            
            response = self.bedrock_runtime.invoke_model(
                modelId=self.model_id,
                body=body
            )
            
            response_body = json.loads(response['body'].read())
            
            if "claude" in self.model_id:
                return response_body['content'][0]['text']
            else:
                return response_body['results'][0]['outputText']
                
        except Exception as e:
            return f"Error invoking model: {e}"
    
    def process_task(self, task_data: dict):
        """Process a task - to be overridden by specific agents"""
        raise NotImplementedError("Subclasses must implement process_task")

print("✅ Base agent framework created!")

## 📊 Performance Comparison

In [ ]:
import time

# Test performance of different models
def benchmark_models():
    """Benchmark different Bedrock models"""
    
    test_prompt = "Analyze the technology market opportunities in Japan in 2 paragraphs."
    
    models_to_test = [
        ("Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v1:0"),
        ("Claude 3 Haiku", "anthropic.claude-3-haiku-20240307-v1:0")
    ]
    
    results = []
    
    for model_name, model_id in models_to_test:
        print(f"Testing {model_name}...")
        
        agent = SageMakerBedrockAgent(f"test_{model_name.lower().replace(' ', '_')}", model_id)
        
        start_time = time.time()
        response = agent.invoke_model(test_prompt, max_tokens=500)
        end_time = time.time()
        
        response_time = end_time - start_time
        response_length = len(response) if "Error" not in response else 0
        
        results.append({
            'Model': model_name,
            'Response Time (s)': round(response_time, 2),
            'Response Length': response_length,
            'Success': "Error" not in response
        })
        
        print(f"  ⏱️  Response time: {response_time:.2f}s")
        print(f"  📝 Response length: {response_length} characters")
        print(f"  ✅ Success: {'Yes' if 'Error' not in response else 'No'}")
        print()
    
    return pd.DataFrame(results)

# Run benchmark
print("🏁 Running model performance benchmark...")
benchmark_df = benchmark_models()

print("📊 Benchmark Results:")
print(benchmark_df.to_string(index=False))

## 📈 Visualize Performance

In [ ]:
# Create performance visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Response time comparison
ax1.bar(benchmark_df['Model'], benchmark_df['Response Time (s)'], color=['#FF6B6B', '#4ECDC4'])
ax1.set_title('Model Response Time Comparison')
ax1.set_ylabel('Response Time (seconds)')
ax1.tick_params(axis='x', rotation=45)

# Response length comparison
ax2.bar(benchmark_df['Model'], benchmark_df['Response Length'], color=['#45B7D1', '#96CEB4'])
ax2.set_title('Response Length Comparison')
ax2.set_ylabel('Response Length (characters)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 Performance visualization complete!")

## ✅ Setup Summary

This notebook has:
1. ✅ Installed required packages for Bedrock integration
2. ✅ Configured AWS environment and tested credentials
3. ✅ Listed available Bedrock models
4. ✅ Tested Claude 3 Sonnet for complex analysis
5. ✅ Tested Claude 3 Haiku for fast processing
6. ✅ Created base agent framework for SageMaker
7. ✅ Benchmarked model performance
8. ✅ Visualized performance metrics

**Next Steps:**
- Proceed to Notebook 2: Market Research Agent Implementation
- Use the base framework to build specialized agents
- Test with real market data and scenarios